# À propos de ce livret


Ce livret présente le modèle de factorisation de matrices par noyaux probabilistes (FMNP).

## Recherches liées

- Modèle implémentée dans ce livret
    - Tinghui Zhou, Hanhuai Shan, Arindam Banerjee, Guillermo Sapiro (2012). **Kernelized probabilistic matrix factorization: exploiting graphs and side information.**  


- Autre modèle
    - Pal, Bithika et Mamata Jenamani (2018). **Kernelized Probabilistic Matrix Factorization for Collaborative Filtering: Exploiting Projected User and Item Graph.**



## Sources de dépôts
>[**Dépôt 1**](https://github.com/kastnerkyle/School/blob/master/atpr/matrix_factorization.py) *version python*  
>[**Dépôt 2**](https://github.com/sg3510/al_proj) *version matlab*



## Note sur le cachier
- Dans la méthode implémentée, l'information déjà disponible sous forme de graph peut être utilisée.


**Tâches**
- [ ] Implémenter SGD / GD

## Préparation de l'environnement pour l'utilisation

In [5]:
# Utilités
import os
import numpy as np
import scipy.sparse as sp
import scipy.linalg as sl

# Chargement des données
import scipy.io

# Barre de progression
from tqdm.auto import trange
from tqdm import tqdm_notebook, notebook

# Présentation des résultats
import matplotlib.pyplot as plot
import matplotlib.cm as cm

In [6]:
os.chdir("C:/Users/amass/OneDrive/02_Education/02_Maitrise/Cours/PROJET_MAITRISE/application")

# Données

## Chargement des données

In [96]:
# Chargement du tenseur de données
tensor = scipy.io.loadmat('data/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']

# Chargement de la matrice aléatoire de données
random_matrix = scipy.io.loadmat('data/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']

# Chargement du tenseur aléatoire de données
random_tensor = scipy.io.loadmat('data/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

print(
    "Tensor shape : {:} | Random matrix shape : {:} | Random tensor shape : {:}"
    .format(tensor.shape, random_matrix.shape, random_tensor.shape))

Tensor shape : (214, 61, 144) | Random matrix shape : (214, 61) | Random tensor shape : (214, 61, 144)


## Format des données

In [97]:
petit_donnees = True

# Paramètres du jeu de données tronqué
capteurs = 50
jours = 10
sequences = tensor.shape[2]

matrice = True

if petit_donnees == True:
    tensor = tensor[0:capteurs, 0:jours, 0:sequences]

    random_tensor = random_tensor[0:capteurs, 0:jours, 0:sequences]

print("The tensors shapes are : {:} | {:}".format(tensor.shape,
                                                  random_tensor.shape))

if matrice == True:
    dense_mat = tensor.reshape(
        [tensor.shape[0], tensor.shape[1] * tensor.shape[2]])

    print("The matrix shape is : {:}".format(dense_mat.shape))

The tensors shapes are : (50, 10, 144) | (50, 10, 144)
The matrix shape is : (50, 1440)


## Scénario de manquants

In [98]:
missing_rate = 0.4

random_missing = True

# Revoir le code pour que les deux choix puissent être respectés sur l'ensemble des matrices/ tenseurs
if random_missing == True:
    print("Random missing (RM) scenario")
    binary_mat = (np.round(random_tensor + 0.5 - missing_rate).reshape([
        random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]
    ]))
    print("\nBinary matrix shape : {:}".format(binary_mat.shape))
else:
    print("Non-random missing (NM) scenario")
    binary_tensor = np.zeros(tensor.shape)
    for i1 in range(tensor.shape[0]):
        for i2 in range(tensor.shape[1]):
            binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 -
                                                missing_rate)
    binary_mat = binary_tensor.reshape([
        binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]
    ])
    print("\nBinary matrix shape : {:}".format(binary_mat.shape))

sparse_mat = np.multiply(dense_mat, binary_mat)

Random missing (RM) scenario

Binary matrix shape : (50, 1440)


## Création du noyau

*Sans informations supplémentaire sur les données, le noyau gaussien est utilisé*

# Modélisation

## Spécification des modèles

Source pour le kernel :
https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy

In [6]:
import numpy as np
import scipy.stats as st

def gkern(kernlen=21, nsig=3):
    """Returns a 2D Gaussian kernel."""

    x = np.linspace(-nsig, nsig, kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    kern2d = kern2d/kern2d.sum()
    
    return kern2d

In [7]:
# Deux boucles d'entrainement ensemble

def KPMF(mat_minus, rang = 10, iterations = 3, segma = 0.2, learning_rate = 0.1):
    
    R = mat_minus # à enlever
    # Identifier les valeurs manquantes
    bin_mat = np.asarray(mat_minus > 0, dtype = np.float64)
    
    N = mat_minus.shape[0]
    M = mat_minus.shape[1]
    np.random.seed(2020) # Reproductibilité
    U = np.random.randn(N, rang)
    np.random.seed(2020) # Reproductibilité
    V = np.random.randn(M, rang)
    
    UV = np.matmul(U, V.T)
    
    segma = segma ** 2
    
    # Noyau V
    SV = np.linalg.pinv(gkern(M))
    # Noyau U
    SU = np.linalg.pinv(gkern(N))
    
    # Initialisation des matrices à incrémenter
    v_t = np.zeros((M, rang))
    u_t = np.zeros((N, rang))
    
    for i in notebook.tqdm(range(iterations)):
        # Gradient sur U
        ## Initialisation de U
        u_t = np.zeros((N, rang))

        for d in notebook.tqdm(range(rang), desc="Rang"):
            for n in notebook.tqdm(range(N), desc="Dimension N"):
                for m in notebook.tqdm(range(M), desc="Dimension M"):
                    # Première partie de l'équation
                    p = -segma * bin_mat[n, m] * (R[n, m] - UV[n, m]) * V[m, d]

                    # Deuxième partie de l'équation
                    ## Calcul de e
                    e = np.zeros((N))
                    np.add.at(e, n, 1)

                    s = e * SU * U[:, d]

                    # Première et deuxième partie de l'équation
                    ens = p + s

                    # Mise à jour
                    # Deuxième partie de la mise à jour
                    app = learning_rate * ens
                    u_t[n, d] += U[n, d] - app[n, d]

        # Mise à jour U
        U = u_t

        # Gradient sur V
        ## Initialisation de V
        v_t = np.zeros((M, rang))

        for d in notebook.tqdm(range(rang), desc="Rang"):
            for n in notebook.tqdm(range(N), desc="Dimension N"):
                for m in notebook.tqdm(range(M), desc="Dimension M"):
                    # Première partie de l'équation
                    p = -segma * bin_mat[n, m] * (R[n, m] - UV[n, m]) * U[n, d]

                    # Deuxième partie de l'équation
                    ## Calcul de e
                    e = np.zeros((M))
                    np.add.at(e, m, 1)

                    s = e * SV * V[:, d]

                    # Première et deuxième partie de l'équation
                    ens = p + s

                    # Mise à jour
                    # Deuxième partie de la mise à jour
                    app = learning_rate * (s + p)
                    v_t[m, d] += V[m, d] - app[m, d]

        # Mise à jour V
        V = v_t
    return U, V

## Entrainement du modèle

In [8]:
# Algorithme KPMF
U, V = KPMF(sparse_mat)

# Matrice avec les facteurs latents
mat_imput = np.dot(U, V.T)

# Présentation des résultats

In [9]:
def get_RMSE(A, A_):
    """
    A : Matrice originale
    A_ : Matrice imputée
    
    
    """
    A1d = np.ravel(A)
    A_1d = np.ravel(A_)
    e = np.mean((A1d - A_1d)**2)
    return np.sqrt(e)

In [10]:
get_RMSE(dense_mat, mat_imput)

2.443995635124845e+79

In [11]:
def combine(A, A_):
    """
    Fonction qui combine le jeu de données avec des données manquantes et l'imputation 
    
    Paramètres en entrée
        A : Matrice avec des données manquantes
        A_ : Matrice avec des données imputées
    Paramètres en sortie
        out : Matrice complétée 
    
    """
    
    
    out = np.zeros(A.shape)
    N = A.shape[0]
    M = A.shape[1]
    for i in range(N):
        for j in range(M):
            if A[i, j] == 0:
                out[i, j] = A_[i, j]
            else:
                out[i, j] = A[i, j]
    return out

In [12]:
mat_combine = combine(sparse_mat, mat_imput)

get_RMSE(dense_mat, mat_combine)

1.826058399206753e+64